In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # I use a HiDPI screen

import os
import numpy as np
import matplotlib.pyplot as plt
import file_loading
from scipy.interpolate import Rbf
from multiprocessing import Pool
from astropy.io import ascii

from forecast import build_interp_zack
from forecast import findlevel
from forecast import plot_cube

In [2]:
nres = 50
computing_PS_flag = True
computing_PC_flag = False

In [3]:
%%time
output_cube_name = 'fid0'
data_dir = '/home/zequnl/Projects/neutrino_mpk/'
table = ascii.read('parameters.table')
fid_num = 0


params, obsarr_PS, obsarr_PC, ell, kappa, invcov_PS, invcov_PC = \
                file_loading.get_data_arrays_across_redshifts( table,
                    data_dir = data_dir,
                    PS_directory = data_dir + 'powerspectrum_ngal40', 
                    PC_directory = data_dir + 'peakcounts_ngal40',
                    redshifts = ['10'])

# we overwrite the inverse covariance matrix with the new one
# NOTE: ngal40 requires a special string!
# invcov_PC = file_loading.compute_custom_PC_cov(  ['10'], 
#      fid_string='Om0.29997_As2.10000_mva0.00000_mvb0.00000_mvc0.00000_h0.70000_Ode0.69995_PC_S_z10_cov_ngal40.npy')

    
# set up grid
x0, y0, z0 = params[fid_num]
x_PS = np.linspace(x0 - 0.04,x0 + 0.04, nres) # M_nu
y_PS = np.linspace(y0 - 0.012, y0 + 0.012, nres) # omega_m
z_PS = np.linspace(z0 - 0.012, z0 + 0.012, nres) # sigma_8
X_PS, Y_PS, Z_PS = np.meshgrid(x_PS, y_PS, z_PS, indexing='ij')
input_param_list_PS = list(zip(np.ravel(X_PS), np.ravel(Y_PS), np.ravel(Z_PS)))

x_PC = np.linspace(x0 - 0.01, x0 + 0.01, nres) # M_nu
y_PC = np.linspace(y0 - 0.003, y0 + 0.003, nres) # omega_m
z_PC = np.linspace(z0 - 0.005, z0 + 0.005, nres) # sigma_8
X_PC, Y_PC, Z_PC = np.meshgrid(x_PC, y_PC, z_PC, indexing='ij')
input_param_list_PC = list(zip(np.ravel(X_PC), np.ravel(Y_PC), np.ravel(Z_PC)))

# ----- BOILER PLATE STUFF BEGINS HERE

# set up interpolator
interp_PS = build_interp_zack(obsarr_PS, params)
interp_PC = build_interp_zack(obsarr_PC, params)

def P_PS( parameter_input ):
    my_interp = interp_PS;
    my_invcov = invcov_PS
    my_fid_model = obsarr_PS[fid_num]
    
    dm = my_fid_model - my_interp( parameter_input ) # d - mu
    return np.exp( -0.5 * np.dot(dm.T,np.dot(my_invcov,dm)) )

def P_PC( parameter_input ):
    my_interp = interp_PC
    my_invcov = invcov_PC
    my_fid_model = obsarr_PC[fid_num]
    
    dm = my_fid_model - my_interp( parameter_input ) # d - mu
    return np.exp( -0.5 * np.dot(dm.T,np.dot(my_invcov,dm)) )

if __name__ == '__main__':
    if computing_PS_flag:
        pool = Pool(16)
        PS_cube = np.array(pool.map(P_PS, input_param_list_PS))
        PS_cube = PS_cube.reshape(X_PS.shape)
        pool.close()
        np.save( 'cubes/' + output_cube_name + '_PS.npy', PS_cube )
        np.save( 'cubes/' + output_cube_name + '_PS_xyz.npy', np.array([x_PS,y_PS,z_PS]) )
        
    if computing_PC_flag:
        pool = Pool(16)
        PC_cube = np.array(pool.map(P_PC, input_param_list_PC))
        PC_cube = PC_cube.reshape(X_PC.shape)
        pool.close()
        np.save( 'cubes/' + output_cube_name + '_PC.npy', PC_cube )
        np.save( 'cubes/' + output_cube_name + '_PC_xyz.npy', np.array([x_PC,y_PC,z_PC]) )

PS bins [24]
PC bins [184]
PS nr 1000 nb 24 0.974974974974975
PC nr 1000 nb 184 0.8148148148148148


ValueError: input data must be at least 2-D

In [ ]:
%%time
output_cube_name = 'fid1'
data_dir = '/home/zequnl/Projects/neutrino_mpk/'
table = ascii.read('parameters.table')
fid_num = 1


params, obsarr_PS, obsarr_PC, ell, kappa, invcov_PS, invcov_PC = \
                file_loading.get_data_arrays_across_redshifts( table,
                    data_dir = data_dir,
                    PS_directory = data_dir + 'powerspectrum_ngal40', 
                    PC_directory = data_dir + 'peakcounts_ngal40',
                    redshifts = ['10'])

# we overwrite the inverse covariance matrix with the new one
# NOTE: ngal40 requires a special string!
# invcov_PC = file_loading.compute_custom_PC_cov(  ['10'], 
#      fid_string='Om0.29997_As2.10000_mva0.00000_mvb0.00000_mvc0.00000_h0.70000_Ode0.69995_PC_S_z10_cov_ngal40.npy')

    
# set up grid
x0, y0, z0 = params[fid_num]
x_PS = np.linspace(x0 - 0.04,x0 + 0.04, nres) # M_nu
y_PS = np.linspace(y0 - 0.012, y0 + 0.012, nres) # omega_m
z_PS = np.linspace(z0 - 0.012, z0 + 0.012, nres) # sigma_8
X_PS, Y_PS, Z_PS = np.meshgrid(x_PS, y_PS, z_PS, indexing='ij')
input_param_list_PS = list(zip(np.ravel(X_PS), np.ravel(Y_PS), np.ravel(Z_PS)))

x_PC = np.linspace(x0 - 0.01, x0 + 0.01, nres) # M_nu
y_PC = np.linspace(y0 - 0.003, y0 + 0.003, nres) # omega_m
z_PC = np.linspace(z0 - 0.005, z0 + 0.005, nres) # sigma_8
X_PC, Y_PC, Z_PC = np.meshgrid(x_PC, y_PC, z_PC, indexing='ij')
input_param_list_PC = list(zip(np.ravel(X_PC), np.ravel(Y_PC), np.ravel(Z_PC)))

# ----- BOILER PLATE STUFF BEGINS HERE

# set up interpolator
interp_PS = build_interp_zack(obsarr_PS, params)
interp_PC = build_interp_zack(obsarr_PC, params)

def P_PS( parameter_input ):
    my_interp = interp_PS;
    my_invcov = invcov_PS
    my_fid_model = obsarr_PS[fid_num]
    
    dm = my_fid_model - my_interp( parameter_input ) # d - mu
    return np.exp( -0.5 * np.dot(dm.T,np.dot(my_invcov,dm)) )

def P_PC( parameter_input ):
    my_interp = interp_PC
    my_invcov = invcov_PC
    my_fid_model = obsarr_PC[fid_num]
    
    dm = my_fid_model - my_interp( parameter_input ) # d - mu
    return np.exp( -0.5 * np.dot(dm.T,np.dot(my_invcov,dm)) )

if __name__ == '__main__':
    if computing_PS_flag:
        pool = Pool(16)
        PS_cube = np.array(pool.map(P_PS, input_param_list_PS))
        PS_cube = PS_cube.reshape(X_PS.shape)
        pool.close()
        np.save( 'cubes/' + output_cube_name + '_PS.npy', PS_cube )
        np.save( 'cubes/' + output_cube_name + '_PS_xyz.npy', np.array([x_PS,y_PS,z_PS]) )
        
    if computing_PC_flag:
        pool = Pool(16)
        PC_cube = np.array(pool.map(P_PC, input_param_list_PC))
        PC_cube = PC_cube.reshape(X_PC.shape)
        pool.close()
        np.save( 'cubes/' + output_cube_name + '_PC.npy', PC_cube )
        np.save( 'cubes/' + output_cube_name + '_PC_xyz.npy', np.array([x_PC,y_PC,z_PC]) )

In [ ]:
%%time
output_cube_name = 'fid2'
data_dir = '/home/zequnl/Projects/neutrino_mpk/'
table = ascii.read('parameters.table')
fid_num = 2


params, obsarr_PS, obsarr_PC, ell, kappa, invcov_PS, invcov_PC = \
                file_loading.get_data_arrays_across_redshifts( table,
                    data_dir = data_dir,
                    PS_directory = data_dir + 'powerspectrum_ngal40', 
                    PC_directory = data_dir + 'peakcounts_ngal40',
                    redshifts = ['10'])

# we overwrite the inverse covariance matrix with the new one
# NOTE: ngal40 requires a special string!
# invcov_PC = file_loading.compute_custom_PC_cov(  ['10'], 
#      fid_string='Om0.29997_As2.10000_mva0.00000_mvb0.00000_mvc0.00000_h0.70000_Ode0.69995_PC_S_z10_cov_ngal40.npy')

    
# set up grid
x0, y0, z0 = params[fid_num]
x_PS = np.linspace(x0 - 0.04,x0 + 0.04, nres) # M_nu
y_PS = np.linspace(y0 - 0.012, y0 + 0.012, nres) # omega_m
z_PS = np.linspace(z0 - 0.012, z0 + 0.012, nres) # sigma_8
X_PS, Y_PS, Z_PS = np.meshgrid(x_PS, y_PS, z_PS, indexing='ij')
input_param_list_PS = list(zip(np.ravel(X_PS), np.ravel(Y_PS), np.ravel(Z_PS)))

x_PC = np.linspace(x0 - 0.01, x0 + 0.01, nres) # M_nu
y_PC = np.linspace(y0 - 0.003, y0 + 0.003, nres) # omega_m
z_PC = np.linspace(z0 - 0.005, z0 + 0.005, nres) # sigma_8
X_PC, Y_PC, Z_PC = np.meshgrid(x_PC, y_PC, z_PC, indexing='ij')
input_param_list_PC = list(zip(np.ravel(X_PC), np.ravel(Y_PC), np.ravel(Z_PC)))

# ----- BOILER PLATE STUFF BEGINS HERE

# set up interpolator
interp_PS = build_interp_zack(obsarr_PS, params)
interp_PC = build_interp_zack(obsarr_PC, params)

def P_PS( parameter_input ):
    my_interp = interp_PS;
    my_invcov = invcov_PS
    my_fid_model = obsarr_PS[fid_num]
    
    dm = my_fid_model - my_interp( parameter_input ) # d - mu
    return np.exp( -0.5 * np.dot(dm.T,np.dot(my_invcov,dm)) )

def P_PC( parameter_input ):
    my_interp = interp_PC
    my_invcov = invcov_PC
    my_fid_model = obsarr_PC[fid_num]
    
    dm = my_fid_model - my_interp( parameter_input ) # d - mu
    return np.exp( -0.5 * np.dot(dm.T,np.dot(my_invcov,dm)) )

if __name__ == '__main__':
    if computing_PS_flag:
        pool = Pool(16)
        PS_cube = np.array(pool.map(P_PS, input_param_list_PS))
        PS_cube = PS_cube.reshape(X_PS.shape)
        pool.close()
        np.save( 'cubes/' + output_cube_name + '_PS.npy', PS_cube )
        np.save( 'cubes/' + output_cube_name + '_PS_xyz.npy', np.array([x_PS,y_PS,z_PS]) )
        
    if computing_PC_flag:
        pool = Pool(16)
        PC_cube = np.array(pool.map(P_PC, input_param_list_PC))
        PC_cube = PC_cube.reshape(X_PC.shape)
        pool.close()
        np.save( 'cubes/' + output_cube_name + '_PC.npy', PC_cube )
        np.save( 'cubes/' + output_cube_name + '_PC_xyz.npy', np.array([x_PC,y_PC,z_PC]) )

In [ ]:
%%time
output_cube_name = 'fid3'
data_dir = '/home/zequnl/Projects/neutrino_mpk/'
table = ascii.read('parameters.table')
fid_num = 3


params, obsarr_PS, obsarr_PC, ell, kappa, invcov_PS, invcov_PC = \
                file_loading.get_data_arrays_across_redshifts( table,
                    data_dir = data_dir,
                    PS_directory = data_dir + 'powerspectrum_ngal40', 
                    PC_directory = data_dir + 'peakcounts_ngal40',
                    redshifts = ['10'])

# we overwrite the inverse covariance matrix with the new one
# NOTE: ngal40 requires a special string!
# invcov_PC = file_loading.compute_custom_PC_cov(  ['10'], 
#      fid_string='Om0.29997_As2.10000_mva0.00000_mvb0.00000_mvc0.00000_h0.70000_Ode0.69995_PC_S_z10_cov_ngal40.npy')

    
# set up grid
x0, y0, z0 = params[fid_num]
x_PS = np.linspace(x0 - 0.04,x0 + 0.04, nres) # M_nu
y_PS = np.linspace(y0 - 0.012, y0 + 0.012, nres) # omega_m
z_PS = np.linspace(z0 - 0.012, z0 + 0.012, nres) # sigma_8
X_PS, Y_PS, Z_PS = np.meshgrid(x_PS, y_PS, z_PS, indexing='ij')
input_param_list_PS = list(zip(np.ravel(X_PS), np.ravel(Y_PS), np.ravel(Z_PS)))

x_PC = np.linspace(x0 - 0.01, x0 + 0.01, nres) # M_nu
y_PC = np.linspace(y0 - 0.003, y0 + 0.003, nres) # omega_m
z_PC = np.linspace(z0 - 0.005, z0 + 0.005, nres) # sigma_8
X_PC, Y_PC, Z_PC = np.meshgrid(x_PC, y_PC, z_PC, indexing='ij')
input_param_list_PC = list(zip(np.ravel(X_PC), np.ravel(Y_PC), np.ravel(Z_PC)))


# ----- BOILER PLATE STUFF BEGINS HERE

# set up interpolator
interp_PS = build_interp_zack(obsarr_PS, params)
interp_PC = build_interp_zack(obsarr_PC, params)

def P_PS( parameter_input ):
    my_interp = interp_PS;
    my_invcov = invcov_PS
    my_fid_model = obsarr_PS[fid_num]
    
    dm = my_fid_model - my_interp( parameter_input ) # d - mu
    return np.exp( -0.5 * np.dot(dm.T,np.dot(my_invcov,dm)) )

def P_PC( parameter_input ):
    my_interp = interp_PC
    my_invcov = invcov_PC
    my_fid_model = obsarr_PC[fid_num]
    
    dm = my_fid_model - my_interp( parameter_input ) # d - mu
    return np.exp( -0.5 * np.dot(dm.T,np.dot(my_invcov,dm)) )

if __name__ == '__main__':
    if computing_PS_flag:
        pool = Pool(16)
        PS_cube = np.array(pool.map(P_PS, input_param_list_PS))
        PS_cube = PS_cube.reshape(X_PS.shape)
        pool.close()
        np.save( 'cubes/' + output_cube_name + '_PS.npy', PS_cube )
        np.save( 'cubes/' + output_cube_name + '_PS_xyz.npy', np.array([x_PS,y_PS,z_PS]) )
        
    if computing_PC_flag:
        pool = Pool(16)
        PC_cube = np.array(pool.map(P_PC, input_param_list_PC))
        PC_cube = PC_cube.reshape(X_PC.shape)
        pool.close()
        np.save( 'cubes/' + output_cube_name + '_PC.npy', PC_cube )
        np.save( 'cubes/' + output_cube_name + '_PC_xyz.npy', np.array([x_PC,y_PC,z_PC]) )

In [ ]:
%%time
output_cube_name = 'fid4'
data_dir = '/home/zequnl/Projects/neutrino_mpk/'
table = ascii.read('parameters.table')
fid_num = 4


params, obsarr_PS, obsarr_PC, ell, kappa, invcov_PS, invcov_PC = \
                file_loading.get_data_arrays_across_redshifts( table,
                    data_dir = data_dir,
                    PS_directory = data_dir + 'powerspectrum_ngal40', 
                    PC_directory = data_dir + 'peakcounts_ngal40',
                    redshifts = ['10'])

# we overwrite the inverse covariance matrix with the new one
# NOTE: ngal40 requires a special string!
# invcov_PC = file_loading.compute_custom_PC_cov(  ['10'], 
#      fid_string='Om0.29997_As2.10000_mva0.00000_mvb0.00000_mvc0.00000_h0.70000_Ode0.69995_PC_S_z10_cov_ngal40.npy')

    
# set up grid
x0, y0, z0 = params[fid_num]
x_PS = np.linspace(x0 - 0.04,x0 + 0.04, nres) # M_nu
y_PS = np.linspace(y0 - 0.012, y0 + 0.012, nres) # omega_m
z_PS = np.linspace(z0 - 0.012, z0 + 0.012, nres) # sigma_8
X_PS, Y_PS, Z_PS = np.meshgrid(x_PS, y_PS, z_PS, indexing='ij')
input_param_list_PS = list(zip(np.ravel(X_PS), np.ravel(Y_PS), np.ravel(Z_PS)))

x_PC = np.linspace(x0 - 0.01, x0 + 0.01, nres) # M_nu
y_PC = np.linspace(y0 - 0.003, y0 + 0.003, nres) # omega_m
z_PC = np.linspace(z0 - 0.005, z0 + 0.005, nres) # sigma_8
X_PC, Y_PC, Z_PC = np.meshgrid(x_PC, y_PC, z_PC, indexing='ij')
input_param_list_PC = list(zip(np.ravel(X_PC), np.ravel(Y_PC), np.ravel(Z_PC)))

# ----- BOILER PLATE STUFF BEGINS HERE

# set up interpolator
interp_PS = build_interp_zack(obsarr_PS, params)
interp_PC = build_interp_zack(obsarr_PC, params)

def P_PS( parameter_input ):
    my_interp = interp_PS;
    my_invcov = invcov_PS
    my_fid_model = obsarr_PS[fid_num]
    
    dm = my_fid_model - my_interp( parameter_input ) # d - mu
    return np.exp( -0.5 * np.dot(dm.T,np.dot(my_invcov,dm)) )

def P_PC( parameter_input ):
    my_interp = interp_PC
    my_invcov = invcov_PC
    my_fid_model = obsarr_PC[fid_num]
    
    dm = my_fid_model - my_interp( parameter_input ) # d - mu
    return np.exp( -0.5 * np.dot(dm.T,np.dot(my_invcov,dm)) )

if __name__ == '__main__':
    if computing_PS_flag:
        pool = Pool(16)
        PS_cube = np.array(pool.map(P_PS, input_param_list_PS))
        PS_cube = PS_cube.reshape(X_PS.shape)
        pool.close()
        np.save( 'cubes/' + output_cube_name + '_PS.npy', PS_cube )
        np.save( 'cubes/' + output_cube_name + '_PS_xyz.npy', np.array([x_PS,y_PS,z_PS]) )
        
    if computing_PC_flag:
        pool = Pool(16)
        PC_cube = np.array(pool.map(P_PC, input_param_list_PC))
        PC_cube = PC_cube.reshape(X_PC.shape)
        pool.close()
        np.save( 'cubes/' + output_cube_name + '_PC.npy', PC_cube )
        np.save( 'cubes/' + output_cube_name + '_PC_xyz.npy', np.array([x_PC,y_PC,z_PC]) )

In [ ]:
%%time
output_cube_name = 'fid5'
data_dir = '/home/zequnl/Projects/neutrino_mpk/'
table = ascii.read('parameters.table')
fid_num = 5


params, obsarr_PS, obsarr_PC, ell, kappa, invcov_PS, invcov_PC = \
                file_loading.get_data_arrays_across_redshifts( table,
                    data_dir = data_dir,
                    PS_directory = data_dir + 'powerspectrum_ngal40', 
                    PC_directory = data_dir + 'peakcounts_ngal40',
                    redshifts = ['10'])

# we overwrite the inverse covariance matrix with the new one
# NOTE: ngal40 requires a special string!
# invcov_PC = file_loading.compute_custom_PC_cov(  ['10'], 
#      fid_string='Om0.29997_As2.10000_mva0.00000_mvb0.00000_mvc0.00000_h0.70000_Ode0.69995_PC_S_z10_cov_ngal40.npy')

    
# set up grid
x0, y0, z0 = params[fid_num]
x_PS = np.linspace(x0 - 0.04,x0 + 0.04, nres) # M_nu
y_PS = np.linspace(y0 - 0.012, y0 + 0.012, nres) # omega_m
z_PS = np.linspace(z0 - 0.012, z0 + 0.012, nres) # sigma_8
X_PS, Y_PS, Z_PS = np.meshgrid(x_PS, y_PS, z_PS, indexing='ij')
input_param_list_PS = list(zip(np.ravel(X_PS), np.ravel(Y_PS), np.ravel(Z_PS)))

x_PC = np.linspace(x0 - 0.01, x0 + 0.01, nres) # M_nu
y_PC = np.linspace(y0 - 0.003, y0 + 0.003, nres) # omega_m
z_PC = np.linspace(z0 - 0.005, z0 + 0.005, nres) # sigma_8
X_PC, Y_PC, Z_PC = np.meshgrid(x_PC, y_PC, z_PC, indexing='ij')
input_param_list_PC = list(zip(np.ravel(X_PC), np.ravel(Y_PC), np.ravel(Z_PC)))


# ----- BOILER PLATE STUFF BEGINS HERE

# set up interpolator
interp_PS = build_interp_zack(obsarr_PS, params)
interp_PC = build_interp_zack(obsarr_PC, params)

def P_PS( parameter_input ):
    my_interp = interp_PS;
    my_invcov = invcov_PS
    my_fid_model = obsarr_PS[fid_num]
    
    dm = my_fid_model - my_interp( parameter_input ) # d - mu
    return np.exp( -0.5 * np.dot(dm.T,np.dot(my_invcov,dm)) )

def P_PC( parameter_input ):
    my_interp = interp_PC
    my_invcov = invcov_PC
    my_fid_model = obsarr_PC[fid_num]
    
    dm = my_fid_model - my_interp( parameter_input ) # d - mu
    return np.exp( -0.5 * np.dot(dm.T,np.dot(my_invcov,dm)) )

if __name__ == '__main__':
    if computing_PS_flag:
        pool = Pool(16)
        PS_cube = np.array(pool.map(P_PS, input_param_list_PS))
        PS_cube = PS_cube.reshape(X_PS.shape)
        pool.close()
        np.save( 'cubes/' + output_cube_name + '_PS.npy', PS_cube )
        np.save( 'cubes/' + output_cube_name + '_PS_xyz.npy', np.array([x_PS,y_PS,z_PS]) )
        
    if computing_PC_flag:
        pool = Pool(16)
        PC_cube = np.array(pool.map(P_PC, input_param_list_PC))
        PC_cube = PC_cube.reshape(X_PC.shape)
        pool.close()
        np.save( 'cubes/' + output_cube_name + '_PC.npy', PC_cube )
        np.save( 'cubes/' + output_cube_name + '_PC_xyz.npy', np.array([x_PC,y_PC,z_PC]) )

In [ ]:
params[0]